In [1]:
import pandas as pd
import numpy as np

In [41]:
df = pd.read_csv('data/mturk_results_0-2000.csv').rename(index=str, columns =
            {'Input.text':'Text', 'Answer.Reaction.on':'Reaction', 
             'Answer.Report.on':'Report', 'Answer.Request.on':'Request',
             'Answer.Urgent.on':'Urgency', 'Answer.NotRelevant.on':'NotRelevant'})
df.pop('Answer.NotUrgent.on')
df['Urgency'] = df['Urgency'].map(lambda x: 1 if x >= 2 else 0)

print(df.shape)
df.head()

(2000, 6)


,Text,Reaction,Report,Request,NotRelevant,Urgency
0,"your rescue boats, vehicles, volunteer craft ...",0,0,1,2,0
1,! @Billcassidy on adding short-term debt limit...,1,1,0,1,0
2,"!! According to #PBS, #Houston convention cen...",0,0,3,0,0
3,"""100,000 homes have been damaged by #Harvey an...",0,3,0,0,0
4,"""Do you need us to get a boat?"" Things I never...",1,0,0,2,0


In [42]:
#now loop through the rows and only keep those whose ties can be broken
relevancy = []
categories = ['NotRelevant', 'Request', 'Report', 'Reaction']
for row in df.itertuples(index=False):
    majority = -1
    for c in categories:
        if getattr(row, c) >= 2:
            majority = categories.index(c)
            break
    relevancy.append(majority)

In [43]:
#now drop all the rows that don't have a majority
df['Relevancy'] = relevancy
df = df[df.Relevancy > -1]
df.drop(categories, axis=1, inplace=True)
print(df.shape)
df.head()

(1741, 3)


,Text,Urgency,Relevancy
0,"your rescue boats, vehicles, volunteer craft ...",0,0
2,"!! According to #PBS, #Houston convention cen...",0,1
3,"""100,000 homes have been damaged by #Harvey an...",0,2
4,"""Do you need us to get a boat?"" Things I never...",0,0
5,"""Flood of epic proportions."" #HurricaneHarvey ...",0,3


In [46]:
df.to_csv('data/mturk_results_0-2000_processed.csv', index=False)